In [2]:
import tabula
import PyPDF2

In [21]:


def extract_tables_from_pdf(file_path):
    # Open the PDF file
    pdf_file = open(file_path, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Get the number of pages in the PDF
    num_pages = len(pdf_reader.pages)

    # Initialize an empty list to store the extracted tables
    tables = []

    # Iterate over each page in the PDF
    for page_num in range(num_pages):
        # Extract the tables from the current page
        page_tables = tabula.read_pdf(file_path, pages=page_num+1, multiple_tables=True)

        # Add the extracted tables to the list
        tables.extend(page_tables)

    # Close the PDF file
    pdf_file.close()

    return tables

# Example usage:
caminho_pdf = './files/old_pdfs/CCI1.088_2024.pdf' # './files/old_pdfs/CCI1.088_2024.pdf'
tables = extract_tables_from_pdf(caminho_pdf)

# Print the extracted tables
for table in tables:
    display(table)

,Data,Descrição,Responsável
0,Até 31/10/2024,Envio da planilha Plano de Ação 2025 por meio ...,Cooperativas centrais
1,Até 5/11/2024,Análiseevalidaçãodasplanilhasrecebidasno\rPort...,CCS


In [22]:
import fitz

doc = fitz.open('./files/old_pdfs/CCI1.088_2024.pdf')
for page in doc:
    tables = page.get_tables()
    for table in tables:
        print(table)

AttributeError: 'Page' object has no attribute 'get_tables'

In [1]:
import ollama

In [15]:
# criar arquivo com os resumos
import json
import os

pasta = './local_storage/'
__arquivos = os.listdir('./local_storage/')
__arqs = sorted(__arquivos)
juncao_textos = []
for __arquivo in __arqs:
    with open(pasta + __arquivo, 'r', encoding='utf-8') as f:
        jsn = json.load(f)
        juncao_textos.append('\n\n' + jsn['page_content'])
# acabou

In [28]:
print(juncao_textos)

[
    '\n\n18/03/2024\r\nNúmero: 0702763-79.2024.8.07.0014\r\nClasse: PROCEDIMENTO COMUM CÍVEL\r\n Órgão julgador: 
Vara Cível do Guará\r\n Última distribuição : 15/03/2024\r\n Valor da causa: R$ 198.133,18\r\n Assuntos: 
Bancários\r\n Nível de Sigilo: 0 (Público)\r\n Justiça gratuita? NÃO\r\n Pedido de liminar ou antecipação de 
tutela? SIM\r\nTribunal de Justiça do Distrito Federal e dos Territórios\r\nPJe - Processo Judicial 
Eletrônico\r\nPartes Advogados\r\nMARISA DA COSTA BAPTISTA (AUTOR)\r\nLUCIANO ALCANTARA BOMM (ADVOGADO)\r\nBANCO 
PAN S.A (REU)\r\nBANCO COOPERATIVO DO BRASIL S/A (REU)\r\nNU PAGAMENTOS S.A. (REU)\r\nEAGLE SOCIEDADE DE CREDITO 
DIRETO S.A. (REU)\r\nDocumentos\r\nId. Data da\r\nAssinatura\r\nDocumento Tipo\r\n190136530 
15/03/2024\r\n14:51\r\nPetição Inicial Petição Inicial\r\n190136532 15/03/2024\r\n14:51\r\n2 Procuração 
Procuração/Substabelecimento\r\n190136533 15/03/2024\r\n14:51\r\n3 Declaração de hipossuficiência Declaração de 
Hipossuficiência\r\n190136536 15/03/2024\r\n14:51\r\n4 Contracheque Outros Documentos\r\n190136539 
15/03/2024\r\n14:51\r\n5 Empréstimo pessoal Documento de Comprovação\r\n190136541 15/03/2024\r\n14:51\r\n6 
Empréstimo consignado Documento de Comprovação\r\n190136543 15/03/2024\r\n14:51\r\n7 Empréstimo consignado 
Documento de Comprovação\r\n190138647 15/03/2024\r\n14:51\r\n8 Extrato bancário Outros Documentos\r\n190138650 
15/03/2024\r\n14:51\r\n9 Extrato Nubank Outros Documentos\r\n190138667 15/03/2024\r\n14:51\r\n10 Documento pessoal 
Documento de Identificação\r\n190207919 15/03/2024\r\n20:27\r\nDecisão Decisão\n',
    '\n\nAssinado eletronicamente por: LUCIANO ALCANTARA BOMM - 15/03/2024 14:49:31 Num. 190136530 - Pág. 
1\r\nhttps://pje.tjdft.jus.br/pje/Processo/ConsultaDocumento/listView.seam?x=24031514493066100000173939548\r\nNúmer
o do documento: 24031514493066100000173939548\r\nEste documento foi gerado pelo usuário 375.***.***-89 em 
18/03/2024 05:30:52\r\nwww.alcantarabomm.com.br\r\nE-mail: contato@alcantarabomm.com.br\r\nTel: (41) 
98830-3893\r\nEXCELENTÍSSIMO SENHOR JUIZ DE DIREITO DA VARA CÍVEL DO FORO \r\nDA COMARCA DE GUARÁ – DISTRITO 
FEDERAL \r\nMARISA DA COSTA BAPTISTA, brasileira, \r\naposentada, portadora da Cédula de Identidade n° 605579, 
inscrita no \r\nCPF/MF sob o nº 225.114.82134, residente e domiciliada na QI 06, \r\nconjunto Q, casa 15 fundos, 
Guará/DF, CEP 71010174 , vem, \r\nrespeitosamente, à presença de Vossa Excelência, por seu advogado, 
\r\ninstrumento de mandato incluso (anexo), com escritório profissional na \r\nRua Euripedes Garcez do Nascimento, 
767, Ahú, Curitiba/PR, onde \r\nrecebe intimações, com fulcro na Lei 14.871/2021 e demais disposições \r\natinentes
à matéria, propor a presente\r\nAÇÃO DE REPACTUAÇÃO DE DÍVIDAS (SUPERENDIVIDAMENTO)\r\ncom pedido liminar 
\r\ndevendo figurar no polo passivo, BANCO PAN \r\nS.A., instituição financeira, inscrita no CNPJ sob o nº 
59.285.411/0001 -\nLUCIANOALCANTARABOMM\nEscritoriodeAdvocacia',
    '\n\nAssinado eletronicamente por: LUCIANO ALCANTARA BOMM - 15/03/2024 14:49:31 Num. 190136530 - Pág. 
2\r\nhttps://pje.tjdft.jus.br/pje/Processo/ConsultaDocumento/listView.seam?x=24031514493066100000173939548\r\nNúmer
o do documento: 24031514493066100000173939548\r\nEste documento foi gerado pelo usuário 375.***.***-89 em 
18/03/2024 05:30:52\r\nwww.alcantarabomm.com.br\r\nE-mail: contato@alcantarabomm.com.br\r\nTel: (41) 
98830-3893\r\n2\r\n13 com sede na Avenida Paulista, nº 1.374 - 12º andar, na Cidade de São \r\nPaulo, Estado de São
Paulo, CEP 01311-300, BANCO COOPERATIVO \r\nSICOOB S.A., instituição financeira inscrita no CNPJ/ME, 
\r\n02.038.232/0001-64, com sede no St de Industrias Gráficas, quadra 06, \r\nn° 2080, Plano Piloto, Brasília/DF, 
CEP 70.610-460, EAGLE SOCIEDADE \r\nDE CREDITO DIRETO S.A, instituição financeira inscrita no CNPJ/ME sob \r\no n° 
45.745.537/0001-19, com sede na Rua Furriel Luiz Antônio de \r\nVargas, n° 250, 14° andar sala A, Bela Vista, Porto
Alegre/RS, CEP \r\n90.

In [ ]:
from app.api.prepdoclib.textparser import TextParser
from app.api.prepdoclib.clean_symbols import CleanSymbolsProcessor
from app.api.prepdoclib.comunicado_splitter import clean_query

parser = TextParser()
clean = CleanSymbolsProcessor()
texto_limpo = parser.parse('\n'.join(juncao_textos))
texto_limpo = parser.clean_empty_lines_mail_sites(texto_limpo)
texto_limpo = parser.cleanup_data(texto_limpo)
texto_limpo = clean.process_line(texto_limpo)
texto_limpo = clean_query(texto_limpo)
print(texto_limpo)

In [84]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama, OllamaEmbeddings
from rich import print
from langchain.globals import set_debug
from langchain_core.pydantic_v1 import BaseModel, Field
from typing_extensions import Annotated, TypedDict, Optional
from langchain_core.runnables import RunnablePassthrough
import json

set_debug(False)
    
model = ChatOllama(model="gemma2:2b-instruct-q4_K_M", keep_alive='1h', temperature=0)

with open('./files/pdfs/0010.txt', 'r', encoding='utf-8') as f:
    texto_limpo = f.read()
    texto_limpo = texto_limpo.split('\n\n\n\n\n')
    texto_limpo = texto_limpo[:4]

    response_schemas = [
        ResponseSchema(
            name="tipo_processo_classe_ou_classe_judicial",
            description="tipo_processo_ou_classe_classe_judicial usado para responder à pergunta do usuário, deve ser o Tipo de Processo, Classe ou Classe Judicial no processo.",
            type='string | \"não encontrado\"'
        ),
        ResponseSchema(
            name="estado_comarca",
            description="estado_comarca usado para responder à pergunta do usuário, deve ser o Estado ou Comarca no processo.",
            type='string | \"não encontrado\"'
        ),
        ResponseSchema(
            name="classificacao_de_prioridade",
            description="classificacao_de_prioridade usado para responder à pergunta do usuário, deve ser a Classificação de Prioridade no processo.",
            type='string | \"não encontrado\"'
        ),
        ResponseSchema(
            name="valor_da_causa",
            description="valor_da_causa usado para responder à pergunta do usuário, deve ser o Valor da causa no processo.",
            type='float | 0.0'
        ),
        ResponseSchema(
            name="orgao_julgador",
            description="orgao_julgador usado para responder à pergunta do usuário, deve ser órgão julgado do processo.",
            type="string | \"não encontrado\""
        ),
        ResponseSchema(
            name="numero_processo",
            description="numero_processo usado para responder à pergunta do usuário, deve ser um Número do processo.",
            type="string | \"não encontrado\""
        ),
        ResponseSchema(
            name="assuntos",
            description="assuntos usado para responder à pergunta do usuário, deve ser um Assunto no processo.",
            type="string | \"não encontrado\""
        ),
        ResponseSchema(
            name="nivel_de_sigilo",
            description="nivel_de_sigilo usado para responder à pergunta do usuário, deve ser o Nível de Sigilo do processo.",
            type="string | \"não encontrado\""
        ),
        ResponseSchema(
            name="justica_gratuita",
            description="justica_gratuita usado para responder à pergunta do usuário, se é gratuita a justiça no processo.",
            type="string | \"não encontrado\""
        ),
        ResponseSchema(
            name="juizes",
            description="juizes usado para responder à pergunta do usuário, os nomes dos juízes descritos no processo.",
            type="string | \"não encontrado\""
        ),
        ResponseSchema(
            name="partes",
            description="partes usado para responder à pergunta do usuário, deve ser as partes do processo.",
            type="List[string] | []"
        ),
        ResponseSchema(
            name="fases_do_processo",
            description="fases_do_processo usado para responder à pergunta do usuário, deve ser as fases processo passou.",
            type="List[string] | []"
        ),
        ResponseSchema(
            name="renda_liquida_mensal",
            description="renda_liquida_mensal usado para responder à pergunta do usuário, deve ser a Renda líquida mensal da parte autora no processo.",
            type="float | 0.0"
        ),
        ResponseSchema(
            name="partes",
            description="partes usado para responder à pergunta do usuário, deve ser as partes do processo.",
            type="List[string] | []"
        ),
        ResponseSchema(
            name="documentos",
            description="documentos usados para responder à pergunta do usuário, devem ser os documentos do processo.",
            type="List[string] | []"
        ),
    ]
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

    format_instructions = output_parser.get_format_instructions(only_json=True)
    lista_jsons = []

    for texto in texto_limpo:
        prompt = PromptTemplate(
            template="Você é um assistente jurídico prestativo, respeitoso e honesto. Sua tarefa é auxiliar os advogados na análise de ações e processos.\n{format_instructions}\n{question}\n\n\n### Contexto ###\n\n```{context}```",
            input_variables=["question"],
            partial_variables={"format_instructions": format_instructions, "context": texto},
        )
        chain = prompt | model
        output = chain.invoke({"question": "Procure no arquivo as informações abaixo de forma precisa e confiável."})
        lista_jsons.append(output.content)


In [85]:
print(lista_jsons)

[
    '```json\n{\n\t"tipo_processo_classe_ou_classe_judicial": "Procedimento Comum Cível",\n\t"estado_comarca": 
"Distrito Federal",\n\t"classificacao_de_prioridade": "Não Informada",\n\t"valor_da_causa": 
198133.18,\n\t"orgao_julgador": "Vara Cível do Guará",\n\t"numero_processo": 
"0702763-79.2024.8.07.0014",\n\t"assuntos": "Bancários",\n\t"nivel_de_sigilo": "0 
(Público)",\n\t"justica_gratuita": "NÃO",\n\t"juizes": "",\n\t"partes": ["MARISA DA COSTA BAPTISTA", "BANCO PAN 
S.A", "BANCO COOPERATIVO DO BRASIL S/A", "NU PAGAMENTOS S.A.", "EAGLE SOCIEDADE DE CREDITO DIRETO 
S.A."],\n\t"fases_do_processo": ["Pedido de liminar ou antecipação de tutela"],\n\t"renda_liquida_mensal": 
0.0,\n\t"partes": ["MARISA DA COSTA BAPTISTA", "BANCO PAN S.A", "BANCO COOPERATIVO DO BRASIL S/A", "NU PAGAMENTOS 
S.A.", "EAGLE SOCIEDADE DE CREDITO DIRETO S.A."],\n\t"documentos": ["Petição Inicial", "Procuração", "Declaração de
hipossuficiência", "Contracheque", "Empréstimo pessoal", "Empréstimo consignado", "Empréstimo consignado", "Extrato
bancário", "Extrato Nubank", "Documento pessoal", "Decisão"]\n}\n``` \n\n\n',
    '```json\n{\n\t"tipo_processo_classe_ou_classe_judicial": "Ação de Representação de 
Dívidas",\n\t"estado_comarca": "Distrito Federal",\n\t"classificacao_de_prioridade": "Não 
Informada",\n\t"valor_da_causa": 0.0,\n\t"orgao_julgador": "Tribunal de Justiça do Distrito 
Federal",\n\t"numero_processo": "Não Informado",\n\t"assuntos": "Superendividamento",\n\t"nivel_de_sigilo": "Não 
Informada",\n\t"justica_gratuita": "Não Informada",\n\t"juizes": "Não Informado",\n\t"partes": ["MARISA DA COSTA 
BAPTISTA", "BANCO PAN S.A."],\n\t"fases_do_processo": ["Início"],\n\t"renda_liquida_mensal": 0.0,\n\t"partes": 
["MARISA DA COSTA BAPTISTA", "BANCO PAN S.A."],\n\t"documentos": ["Ação de Representação de Dívidas 
(Superendividamento)"]\n}\n```',
    '```json\n{\n\t"tipo_processo_classe_ou_classe_judicial": "Processo Civil",\n\t"estado_comarca": "São 
Paulo",\n\t"classificacao_de_prioridade": "Não Informada",\n\t"valor_da_causa": 3110.6,\n\t"orgao_julgador": 
"Tribunal de Justiça do Estado de São Paulo",\n\t"numero_processo": "24031514493066100000173939548",\n\t"assuntos":
"Superendividamento",\n\t"nivel_de_sigilo": "Não Informado",\n\t"justica_gratuita": "Sim",\n\t"juizes": "Não 
Informado",\n\t"partes": ["LUCIANO ALCANTARA BOMM"],\n\t"fases_do_processo": 
["Inicial"],\n\t"renda_liquida_mensal": 3110.6,\n\t"partes": ["LUCIANO ALCANTARA BOMM"],\n\t"documentos": 
[]\n}\n```',
    '```json\n{\n\t"tipo_processo_classe_ou_classe_judicial": "Processo Civil",\n\t"estado_comarca": "não 
encontrado",\n\t"classificacao_de_prioridade": "não encontrado",\n\t"valor_da_causa": 0.0,\n\t"orgao_julgador": 
"não encontrado",\n\t"numero_processo": "24031514493066100000173939548",\n\t"assuntos": "não 
encontrado",\n\t"nivel_de_sigilo": "não encontrado",\n\t"justica_gratuita": "não encontrado",\n\t"juizes": "não 
encontrado",\n\t"partes": ["LUCIANO ALCANTARA BOMM"],\n\t"fases_do_processo": ["não 
encontrado"],\n\t"renda_liquida_mensal": 0.0,\n\t"partes": ["LUCIANO ALCANTARA BOMM"],\n\t"documentos": 
["anexo"]\n}\n``` \n\n\n**Análise:**\n\nO texto fornecido é um trecho de um documento eletrônico assinado por 
Luciano Alcantara Bomm. O documento descreve a situação financeira da parte autora, que possui dívidas superiores à
sua renda mensal.  \n\n**Informações relevantes:**\n\n* **Tipo de processo:** Processo Civil\n* **Parte autora:** 
Luciano Alcantara Bomm \n* **Documento:** Anexos ao documento original.\n\n\n**Observações:**\n\n* O contexto do 
documento não fornece informações sobre o estado da comarca, a classificação de prioridade, o valor da causa ou os 
juízes envolvidos no processo. \n* A informação sobre as partes e as fases do processo é ausente. \n* A renda 
líquida mensal da parte autora é desconhecida. \n\n\n'
]

In [ ]:
"You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",


            template = """<s>[INST] You are a helpful, respectful and honest legal assistant.
Your task is to assist lawyers in legal case discovery.
Answer the question below from context below.
{context}
{question} [/INST] </s>
"""